# TopAGNPS run sequence

- This notebook executes the commands to:
    - Run the preprocessing step of TopAGNPS from presribed boundary
    - Identify the outlet
    - Finish the computation

In [1]:
import sys
sys.path.append('..')
import geopandas as gpd

from pyagnps import topagnps


## Define paths and select geometry for DEM selection

In [2]:
path_to_thucs = '../../inputs/thucs/tophuc_S_40000.gpkg'
root_dir = '../../outputs/topagnps/'
# path_to_input_raster = '../../inputs/TNM_DEM/USGS_13_n41w092_20220303.tif'
# path_to_input_raster = '../../inputs/TNM_DEM/HRNHDPlusRasters0708/hydrodem.tif'
path_to_input_raster = 'D:/AIMS/NHDPlusHR_raster_tests/hydrodem_shifted_98966_cm_unwalled_4800_m_topagnps.tif'
# path_to_input_raster = 'D:/AIMS/NHDPlusHR_raster_tests/hydrodem_shifted_98867_cm_unwalled_topagnps_meters_float32.tif'

In [3]:
thucs = gpd.read_file(path_to_thucs)

In [38]:
thuc_id = '1175'
# thucid_dir_name = f'thuc_{thuc_id}'
thucid_dir_name = f'thuc_{thuc_id}_hydrodem_shifted_98966_cm_unwalled_4800_m_test'
thuc_select = thucs[thucs['tophucid']==thuc_id]
thuc_select

,tophucid,drainage_area_sqkm,outlethuc12,outlethuc12_hutype,receivinghuc12,contained_hucs,num_contained_hucs,foreign_inflows,num_foreign_inflows,totophuc,bbox_area_sqkm,tophucid_bbox_container,geometry
1167,1175,1394.19,070801041806,S,071100010601,"070801041704,070801041701,070801041803,0708010...",12,"070801041302,070801041605,070801071006,0708010...",7,1177,2802.375316,None,"POLYGON ((-91.28058 40.38634, -91.28122 40.385..."


## Download/Read DEM and write to `.asc` file

In [40]:
import importlib
importlib.reload(topagnps);

path_to_dir = topagnps.create_topagnps_directory(root_dir, thucid_dir_name)

# dem, path_to_asc = topagnps.download_dem(thuc_select, path_to_dir, name=thucid_dir_name, resolution_m=10, buffer_m=500)
dem, path_to_asc = topagnps.get_dem_from_raster(thuc_select, path_to_input_raster, path_to_dir, name=thucid_dir_name, resolution_m=10, buffer_m=500, keeptif=True)

c:\Users\Luc\anaconda3\envs\pyagnps\lib\site-packages\rasterio\windows.py:310: RasterioDeprecationWarning: The height, width, and precision parameters are unused, deprecated, and will be removed in 2.0.0.
  warnings.warn(


nan
<class 'numpy.float64'>


In [33]:
dem.rio.nodata

-2147483648.0

## Define `TOPAGNPS.XML` control file

Define the parameters for the preprocessing step

In [34]:
dem_filename = path_to_asc.rsplit('/',1)[-1] # Part of the string after the last / = "thuc_1173_rest_10_m.asc"

topagnpsXML = {'DEMPROC': 2,
               'FORMAT': 0,
               'CSA': 10,
               'MSCL': 250,
               'KEEPFILES': 1,
               'FILENAME': dem_filename}

topagnps.create_topagnps_xml_control_file(topagnpsXML, path_to_dir+'/TOPAGNPS.XML')

## Run TopAGNPS (pre-processing step)

In [35]:
path_to_TOPAGNPS_bin = '../../../src/bins/TopAGNPS_v6.00.b.017_release_64-bit_Windows.exe' # absolute or with respect to a sub directory in path_to_dir

topagnps.run_topagnps(path_to_dir, path_to_TOPAGNPS_bin)

## Find outlet using `UPAREA.ASC` file and the t-huc geometry

In [36]:
xout, yout, rowout, colout, raster_crs = topagnps.find_outlet_uparea_shape_intersection(path_to_dir+'/UPAREA.ASC', thuc_select)
print(xout, yout, rowout, colout)
print(raster_crs)

385430.0 1938660.0 4754 1068
ESRI:102039


## Finish computation

In [37]:
topagnpsXML = {'DEMPROC': 0,
               'FORMAT': 0,
               'CSA': 10,
               'MSCL': 250,
               'KEEPFILES': 1,
               'OUTROW': rowout,
               'OUTCOL': colout,
               'READOUT': 1,
               'FILENAME': dem_filename}

topagnps.create_topagnps_xml_control_file(topagnpsXML, path_to_dir+'/TOPAGNPS.XML')
topagnps.run_topagnps(path_to_dir, path_to_TOPAGNPS_bin)

In [65]:
import pandas as pd

df = pd.read_csv('C:/Users/Luc/projects/pyagnps/inputs/thucs/lmrb.csv')

In [68]:
df = df[['tophucid']]

In [70]:
df.loc[df['tophucid']==521,'done']=True

In [91]:
import pyagnps.utils as utils

import importlib

importlib.reload(utils);

In [ ]:
keep_files =['AgFlow_LS_Factor.asc',
             'AgFlow_LS_Factor.prj',
             'AnnAGNPS_Cell_IDs.asc',
             'AnnAGNPS_Cell_IDs.prj',
             'AnnAGNPS_Reach_IDs.asc',
             'AnnAGNPS_Reach_IDs.prj',
             'FLOVEC.ASC',
             'FLOVEC.PRJ',
             'NETFUL.ASC',
             'NETFUL.PRJ',
             'UPAREA.ASC',
             'UPAREA.PRJ',
             'AgFlow_Reach_Data.csv',
             'AnnAGNPS_Cell_Data_Section.csv',
             'AnnAGNPS_Reach_Data_Section.csv',
             'TopAGNPS_log.CSV',
             'TopAGNPS_status.CSV',
             'TopAGNPS_wrn.CSV',
             'TOPAGNPS.XML']

path_to_dir = 'C:/Users/Luc/projects/pyagnps/outputs/topagnps/thuc_1169_del'

dem_name = 'thuc_1169_res_10_m'

keep_files.append(f'{dem_name}.asc')
keep_files.append(f'{dem_name}.prj')

file_del_errors = utils.remove_all_files_from_dir_except_from_list(path_to_dir, keep_files)
# need to append dem name